# Train a Model with SageMaker Autopilot

Customer review를 예측하기 위해 SageMaker Autopilot을 사용합니다.  
Autopilot는 AutoML에 대해 white-box 접근 방식으로 구현합니다.

<img src="./img/autopilot.png" width="80%" align="left">

# Introduction

Amazon SageMaker Autopilot은 데이터 세트에서 자동 기계 학습 (AutoML)을 수행하는 서비스입니다. Autopilot은 UI 또는 AWS SDK를 통해 사용할 수 있습니다. 이 노트북에서는 AWS SDK를 사용하여 텍스트 처리 및 sentiment classification 기계 학습 파이프 라인을 생성과 배포를 합니다.

# Setup

* 모델 학습에 사용되는 S3 bucket과 prefix 가 필요합니다. 참고 : 필요한 dataset은 동일 region에 있어야 합니다.
* 이 노트북의 IAM role은 dataset에 액세스가 가능해야 합니다.

In [1]:
%store -r

In [2]:
import boto3
import sagemaker
import pandas as pd

# Execute statement using connection cursor
from pyathena import connect
from pyathena.util import as_pandas

sess   = sagemaker.Session()
sm = boto3.Session().client(service_name='sagemaker', region_name=region_name)

# Dataset

In [3]:
print(header_train_s3_uri)

s3://sagemaker-us-east-1-322537213286/data/amazon_reviews_us_Digital_Software_v1_00_header.csv


In [4]:
!aws s3 ls $header_train_s3_uri

2020-11-18 13:04:10   29818030 amazon_reviews_us_Digital_Software_v1_00_header.csv


# Setup the S3 Location for the Autopilot-Generated Assets 

* Jupyter Notebooks (Analysis)
* Python Scripts (Feature Engineering)
* Trained Models.

In [5]:
prefix_model_output = 'models/autopilot'

model_output_s3_uri = 's3://{}/{}'.format(job_bucket, prefix_model_output)

print(model_output_s3_uri)


s3://sagemaker-experiments-us-east-1-322537213286/models/autopilot


In [6]:
max_candidates = 3

job_config = {
    'CompletionCriteria': {
      'MaxRuntimePerTrainingJobInSeconds': 600,
      'MaxCandidates': max_candidates,
      'MaxAutoMLJobRuntimeInSeconds': 3600
    },
}

input_data_config = [{
      'DataSource': {
        'S3DataSource': {
          'S3DataType': 'S3Prefix',
          'S3Uri': '{}'.format(header_train_s3_uri)
        }
      },
      'TargetAttributeName': 'star_rating'
    }
]

output_data_config = {
    'S3OutputPath': '{}'.format(model_output_s3_uri)
}

# Launch the SageMaker Autopilot job

`create_auto_ml_job` API를 이용하여 Autopilot job을 실행합니다.

In [7]:
from time import gmtime, strftime, sleep
timestamp_suffix = strftime('%d-%H-%M-%S', gmtime())

auto_ml_job_name = 'automl-dm-' + timestamp_suffix
print('AutoMLJobName: ' + auto_ml_job_name)

AutoMLJobName: automl-dm-19-01-33-55


`ProblemType`를 특정할 수 없는 경우에는 Autopilot이 자동으로 regression 또는 Classification (binary 또는 multi-class) 를 탐지합니다.

In [8]:
sm.create_auto_ml_job(AutoMLJobName=auto_ml_job_name,
                      InputDataConfig=input_data_config,
                      OutputDataConfig=output_data_config,
                      AutoMLJobConfig=job_config,
#                      ProblemType="Classification",
                      RoleArn=role)

{'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:322537213286:automl-job/automl-dm-19-01-33-55',
 'ResponseMetadata': {'RequestId': '4e0f3da0-0fe9-4461-a850-4147f8ce8608',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4e0f3da0-0fe9-4461-a850-4147f8ce8608',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '92',
   'date': 'Thu, 19 Nov 2020 01:33:56 GMT'},
  'RetryAttempts': 0}}

# Tracking the progress of the Autopilot job

SageMaker Autopilot은 high-level 단계로 아래와 같이 구성됩니다.

* _Data Analysis_ 데이터를 요약하고 분석하여 탐색할 feature engineering 기법, 하이퍼파라미터와 탐색 모델을 결정합니다.
* _Feature Engineering_ 데이터가 클린징, 밸런싱, 결합 과 훈련/검증데이터셋으로 분리합니다.
* _Model Training and Tuning_ 가장 높은 성능의 features, 하이퍼라미터와 모델을 선택하고 학습합니다.

# Analyzing Data

In [9]:
# Sleep for a bit to ensure the AutoML job above has time to start
import time
time.sleep(3)

job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']

if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('AnalyzingData'):
        job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(30)
    print("Data analysis complete")
    
print(job)

InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress AnalyzingData
InProgress FeatureEngineering
Data analysis complete
{'AutoMLJobName': 'automl-dm-19-01-33-55', 'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:322537213286:automl-job/automl-dm-19-01-33-55', 'InputDataConfig': [{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-322537213286/data/amazon_reviews_us_Digital_Software_v1_00_header.csv'}}, 'TargetAttributeName': 'star_rating'}], 'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-experiments-us-east-1-32253721

# Feature Engineering

In [10]:
job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']
print(job_status)
print(job_sec_status)
if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('FeatureEngineering'):
        job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(30)
    print("Feature engineering complete")
    
print(job)

InProgress
FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress FeatureEngineering
InProgress ModelTuning
Feature engineering complete
{'AutoMLJobName': 'automl-dm-19-01-33-55', 'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:322537213286:automl-job/automl-dm-19-01-33-55', 'InputDataConfig': [{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-322537213286/data/amazon_reviews_us_Digital_Software_v1_00_header.csv'}}, 'TargetAttributeName': 'star_rating'}], 'OutputDataCon

# Model Training and Tuning

In [11]:
job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
job_status = job['AutoMLJobStatus']
job_sec_status = job['AutoMLJobSecondaryStatus']
print(job_status)
print(job_sec_status)
if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('ModelTuning'):
        job = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)
        job_status = job['AutoMLJobStatus']
        job_sec_status = job['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        sleep(30)
    print("Model tuning complete")
    
print(job)

InProgress
ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
InProgress ModelTuning
Completed MaxCandidatesReached
Model tuning complete
{'AutoMLJobName': 'automl-dm-19-01-33-55', 'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:322537213286:automl-job/automl-dm-19-01-33-55', 'InputDataConfig': [{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-322537213286/data/amazon_reviews_us_Digital_Software_v1_00_header.csv'}}, 'TargetAttributeName': 'star_rating'}], 'OutputDataConfig': {'S3OutputPath': 's3://sagemaker-experiments-us-east-1-322537213286/models/autopilot'}, 'RoleArn': 'arn:aws:iam::322537213286:role/service-role/AIMLWorkshop-targetfor-1124-SageMakerIamRole-TWXQ0O23QRAY', 'AutoMLJobConfig': {'CompletionCriteria': {'MaxCandidates': 3, 'MaxRuntimePerTrainingJobInSeconds': 600, '

<h2><span style="color:red">위 Autopilot이 완료되기 전까지 기다려 주시기 바랍니다.</span></h2>

# View Generated Notebooks

데이터 분석이 완료되며, SageMaker AutoPilot 2가지 노트북을 생성합니다.
* Data exploration,
* Candidate definition.

## Copy the Generated Notebooks Locally

In [41]:
generated_resources = job['AutoMLJobArtifacts']['DataExplorationNotebookLocation'].rstrip('notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb')
generated_resources

's3://sagemaker-experiments-us-east-1-322537213286/models/autopilot/automl-dm-19-01-33-55/sagemaker-automl-candidates/pr-1-5ae06caecb484da2b6ee959a0c0c4f0b7fafdb0242f5473197466f212f'

In [42]:
!aws s3 cp --recursive $generated_resources .

download: s3://sagemaker-experiments-us-east-1-322537213286/models/autopilot/automl-dm-19-01-33-55/sagemaker-automl-candidates/pr-1-5ae06caecb484da2b6ee959a0c0c4f0b7fafdb0242f5473197466f212f/generated_module/README.md to generated_module/README.md
download: s3://sagemaker-experiments-us-east-1-322537213286/models/autopilot/automl-dm-19-01-33-55/sagemaker-automl-candidates/pr-1-5ae06caecb484da2b6ee959a0c0c4f0b7fafdb0242f5473197466f212f/generated_module/candidate_data_processors/sagemaker_serve.py to generated_module/candidate_data_processors/sagemaker_serve.py
download: s3://sagemaker-experiments-us-east-1-322537213286/models/autopilot/automl-dm-19-01-33-55/sagemaker-automl-candidates/pr-1-5ae06caecb484da2b6ee959a0c0c4f0b7fafdb0242f5473197466f212f/generated_module/candidate_data_processors/dpp1.py to generated_module/candidate_data_processors/dpp1.py
download: s3://sagemaker-experiments-us-east-1-322537213286/models/autopilot/automl-dm-19-01-33-55/sagemaker-automl-candidates/pr-1-5ae06c

## In the file view, open the following folders:
```
notebooks/
generated_module/
```
이 폴더에서 많은 정보를 보실 수 있습니다.

# Viewing All Candidates

model tuning이 완료되면 AutoML에서 탐색 한 모든 후보(서로 다른 하이퍼 파라미터 조합을 가진 파이프 라인 평가)를 보고, 최종 성능 메트릭별로 정렬할 수 있습니다.

In [14]:
candidates = sm.list_candidates_for_auto_ml_job(AutoMLJobName=auto_ml_job_name, 
                                                SortBy='FinalObjectiveMetricValue')['Candidates']
for index, candidate in enumerate(candidates):
    print(str(index) + "  " 
        + candidate['CandidateName'] + "  " 
        + str(candidate['FinalAutoMLJobObjectiveMetric']['Value']))

0  tuning-job-1-7a6f5739bfae48ffbf-002-6bf8e438  0.48201999068260193
1  tuning-job-1-7a6f5739bfae48ffbf-001-06837492  0.2903299927711487
2  tuning-job-1-7a6f5739bfae48ffbf-003-e256aa27  0.2791000008583069


# Inspect Trials using Experiments API

SageMaker Autopilot은 자동으로 새로운 experiment를 생성하고, 각 trial에 대한 정보를 experiment에 추가합니다.

In [15]:
from sagemaker.analytics import ExperimentAnalytics, TrainingJobAnalytics

exp = ExperimentAnalytics(
        sagemaker_session=sess, 
        experiment_name=auto_ml_job_name + '-aws-auto-ml-job',
)

df = exp.dataframe()
df

,TrialComponentName,DisplayName,SourceArn,SageMaker.ImageUri,SageMaker.InstanceCount,SageMaker.InstanceType,SageMaker.VolumeSizeInGB,_tuning_objective_metric,alpha,colsample_bytree,...,code - MediaType,code - Value,input_channel_mode,job_name,label_col,max_dataset_size,SageMaker.ImageUri - MediaType,SageMaker.ImageUri - Value,ds - MediaType,ds - Value
0,tuning-job-1-7a6f5739bfae48ffbf-003-e256aa27-a...,tuning-job-1-7a6f5739bfae48ffbf-003-e256aa27-a...,arn:aws:sagemaker:us-east-1:322537213286:train...,683313688378.dkr.ecr.us-east-1.amazonaws.com/s...,1.0,ml.m5.4xlarge,50.0,validation:accuracy,0.000021,0.534112,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tuning-job-1-7a6f5739bfae48ffbf-002-6bf8e438-a...,tuning-job-1-7a6f5739bfae48ffbf-002-6bf8e438-a...,arn:aws:sagemaker:us-east-1:322537213286:train...,683313688378.dkr.ecr.us-east-1.amazonaws.com/s...,1.0,ml.m5.4xlarge,50.0,validation:accuracy,0.000001,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tuning-job-1-7a6f5739bfae48ffbf-001-06837492-a...,tuning-job-1-7a6f5739bfae48ffbf-001-06837492-a...,arn:aws:sagemaker:us-east-1:322537213286:train...,683313688378.dkr.ecr.us-east-1.amazonaws.com/s...,1.0,ml.m5.4xlarge,50.0,validation:accuracy,0.000001,1.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,automl-dm--dpp1-csv-1-b43ef87eb77448e4b67fb8de...,automl-dm--dpp1-csv-1-b43ef87eb77448e4b67fb8de...,arn:aws:sagemaker:us-east-1:322537213286:trans...,NaN,1.0,ml.m5.4xlarge,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,automl-dm--dpp2-rpb-1-a4250ff0fd4e4c2c9180b738...,automl-dm--dpp2-rpb-1-a4250ff0fd4e4c2c9180b738...,arn:aws:sagemaker:us-east-1:322537213286:trans...,NaN,1.0,ml.m5.4xlarge,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,automl-dm--dpp1-1-d6cb1b2cef4d4ba9a6de2c757f02...,automl-dm--dpp1-1-d6cb1b2cef4d4ba9a6de2c757f02...,arn:aws:sagemaker:us-east-1:322537213286:train...,683313688378.dkr.ecr.us-east-1.amazonaws.com/s...,1.0,ml.m5.4xlarge,50.0,NaN,NaN,NaN,...,application/x-code,s3://sagemaker-experiments-us-east-1-322537213...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,automl-dm--dpp2-1-7a6b7dc133994fd3abef6279c934...,automl-dm--dpp2-1-7a6b7dc133994fd3abef6279c934...,arn:aws:sagemaker:us-east-1:322537213286:train...,683313688378.dkr.ecr.us-east-1.amazonaws.com/s...,1.0,ml.m5.4xlarge,50.0,NaN,NaN,NaN,...,application/x-code,s3://sagemaker-experiments-us-east-1-322537213...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,db-1-2d9c527a83a04946aa94e02498351a6c67dba2cd4...,db-1-2d9c527a83a04946aa94e02498351a6c67dba2cd4...,arn:aws:sagemaker:us-east-1:322537213286:proce...,NaN,1.0,ml.m5.2xlarge,250.0,NaN,NaN,NaN,...,NaN,NaN,Pipe,automl-dm-19-01-33-55,star_rating,5,NaN,120479346908.dkr.ecr.us-east-1.amazonaws.com/d...,NaN,s3://sagemaker-experiments-us-east-1-322537213...


# Explore the Best Candidate

Dataset에서 AutoML job을 완료하고 trials를 시각화하면, 단일 API call로 어떤 trials에 대해 모델을 생성할 수 있습니다. [Inference Pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html) 사용하여 온라인으로 또는 batch prediction으로 모델을 배포합니다. 이 노트북에서는 가장 성능이 좋은 trial을 선택하여 inference로 배포합니다.


In [16]:
best_candidate = sm.describe_auto_ml_job(AutoMLJobName=auto_ml_job_name)['BestCandidate']
best_candidate_identifier = best_candidate['CandidateName']

print("Candidate name: " + best_candidate_identifier)
print("Metric name: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("Metric value: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

Candidate name: tuning-job-1-7a6f5739bfae48ffbf-002-6bf8e438
Metric name: validation:accuracy
Metric value: 0.48201999068260193


In [17]:
best_candidate

{'CandidateName': 'tuning-job-1-7a6f5739bfae48ffbf-002-6bf8e438',
 'FinalAutoMLJobObjectiveMetric': {'MetricName': 'validation:accuracy',
  'Value': 0.48201999068260193},
 'ObjectiveStatus': 'Succeeded',
 'CandidateSteps': [{'CandidateStepType': 'AWS::SageMaker::ProcessingJob',
   'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:322537213286:processing-job/db-1-2d9c527a83a04946aa94e02498351a6c67dba2cd48884cd48eff37376e',
   'CandidateStepName': 'db-1-2d9c527a83a04946aa94e02498351a6c67dba2cd48884cd48eff37376e'},
  {'CandidateStepType': 'AWS::SageMaker::TrainingJob',
   'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:322537213286:training-job/automl-dm--dpp2-1-7a6b7dc133994fd3abef6279c934d711de56297fd9f24',
   'CandidateStepName': 'automl-dm--dpp2-1-7a6b7dc133994fd3abef6279c934d711de56297fd9f24'},
  {'CandidateStepType': 'AWS::SageMaker::TransformJob',
   'CandidateStepArn': 'arn:aws:sagemaker:us-east-1:322537213286:transform-job/automl-dm--dpp2-rpb-1-a4250ff0fd4e4c2c9180b738f88ea32b56c

inference 파이프라인은 model과 containers로 구성되어 있습니다.

In [18]:
for container in best_candidate['InferenceContainers']:
    print(container['Image'])
    print(container['ModelDataUrl'])
    print('======================')

683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-sklearn-automl:0.2-1-cpu-py3
s3://sagemaker-experiments-us-east-1-322537213286/models/autopilot/automl-dm-19-01-33-55/data-processor-models/automl-dm--dpp2-1-7a6b7dc133994fd3abef6279c934d711de56297fd9f24/output/model.tar.gz
683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3
s3://sagemaker-experiments-us-east-1-322537213286/models/autopilot/automl-dm-19-01-33-55/tuning/automl-dm--dpp2-xgb/tuning-job-1-7a6f5739bfae48ffbf-002-6bf8e438/output/model.tar.gz
683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-sklearn-automl:0.2-1-cpu-py3
s3://sagemaker-experiments-us-east-1-322537213286/models/autopilot/automl-dm-19-01-33-55/data-processor-models/automl-dm--dpp2-1-7a6b7dc133994fd3abef6279c934d711de56297fd9f24/output/model.tar.gz


# Autopilot Chooses XGBoost as Best Candidate!


Autopilot은 XGBoost 모델의 기본적인 값과 다른 하이퍼파라미터와 feature transformations을 선택합니다.

# Deploy the Model as a REST Endpoint

Batch transformations 또한 지원하지만, 여기서는 REST Endpoint를 생성합니다.

In [19]:
model_name = 'automl-dm-model-' + timestamp_suffix

model_arn = sm.create_model(Containers=best_candidate['InferenceContainers'],
                            ModelName=model_name,
                            ExecutionRoleArn=role)

print('Best candidate model ARN: ', model_arn['ModelArn'])

Best candidate model ARN:  arn:aws:sagemaker:us-east-1:322537213286:model/automl-dm-model-19-01-33-55


In [20]:
# EndpointConfig name
timestamp_sufafix = strftime('%d-%H-%M-%S', gmtime())
epc_name = 'automl-dm-epc-' + timestamp_suffix

# Endpoint name
autopilot_endpoint_name = 'automl-dm-ep-' + timestamp_suffix
variant_name = 'automl-dm-variant-' + timestamp_suffix

print(autopilot_endpoint_name)
print(variant_name)

automl-dm-ep-19-01-33-55
automl-dm-variant-19-01-33-55


In [21]:
ep_config = sm.create_endpoint_config(EndpointConfigName = epc_name,
                                      ProductionVariants=[{'InstanceType':'ml.m5.large',
                                                           'InitialInstanceCount': 1,
                                                           'ModelName': model_name,
                                                           'VariantName': variant_name}])


In [22]:
create_endpoint_response = sm.create_endpoint(EndpointName=autopilot_endpoint_name,
                                              EndpointConfigName=epc_name)
print(create_endpoint_response['EndpointArn'])

arn:aws:sagemaker:us-east-1:322537213286:endpoint/automl-dm-ep-19-01-33-55


# Wait for the Model to Deploy


모델을 deploy하는데에는 5~10분 정도 소요됩니다.

In [23]:
sm.get_waiter('endpoint_in_service').wait(EndpointName=autopilot_endpoint_name)


In [24]:
resp = sm.describe_endpoint(EndpointName=autopilot_endpoint_name)
status = resp['EndpointStatus']

print("Arn: " + resp['EndpointArn'])
print("Status: " + status)

Arn: arn:aws:sagemaker:us-east-1:322537213286:endpoint/automl-dm-ep-19-01-33-55
Status: InService


# Test Our Model with Some Example Reviews
Let's do some ad-hoc predictions on our model.

In [25]:
sm_runtime = boto3.client('sagemaker-runtime')

In [26]:
csv_line_predict_positive = """I loved it!"""

response = sm_runtime.invoke_endpoint(EndpointName=autopilot_endpoint_name, ContentType='text/csv', Accept='text/csv', Body=csv_line_predict_positive)

response_body = response['Body'].read().decode('utf-8').strip()
response_body

'5'

In [27]:
csv_line_predict_meh = """It's OK."""

response = sm_runtime.invoke_endpoint(EndpointName=autopilot_endpoint_name, ContentType='text/csv', Accept='text/csv', Body=csv_line_predict_meh)

response_body = response['Body'].read().decode('utf-8').strip()
response_body

'3'

In [28]:
csv_line_predict_negative = """The worst product ever."""

response = sm_runtime.invoke_endpoint(EndpointName=autopilot_endpoint_name, ContentType='text/csv', Accept='text/csv', Body=csv_line_predict_negative)

response_body = response['Body'].read().decode('utf-8').strip()
response_body

'1'

# Create an Athena Table with Sample Reviews

In [29]:
table_name = 'product_reviews'

In [30]:
# Create Table SQL Statement
statement = """
DROP TABLE IF EXISTS {}.{}
""".format(database_name, table_name, database_name, table_name_tsv)

print(statement)

cursor = connect(region_name=region_name, s3_staging_dir=s3_staging_dir).cursor()
cursor.execute(statement)


DROP TABLE IF EXISTS awsdb_1124.product_reviews



In [31]:
# Create Table SQL Statement
statement = """
CREATE TABLE IF NOT EXISTS {}.{} AS 
SELECT review_id, review_body , star_rating
FROM {}.{}
""".format(database_name, table_name, database_name, table_name_tsv)

print(statement)


CREATE TABLE IF NOT EXISTS awsdb_1124.product_reviews AS 
SELECT review_id, review_body , star_rating
FROM awsdb_1124.amazon_reviews_tsv



In [32]:
# Execute statement using connection cursor
from pyathena import connect
from pyathena.util import as_pandas

cursor = connect(region_name=region_name, s3_staging_dir=s3_staging_dir).cursor()
cursor.execute(statement)

In [33]:
statement = 'SELECT * FROM {}.{} LIMIT 10'.format(database_name, table_name)
cursor.execute(statement)

In [34]:
df_show = as_pandas(cursor)
df_show

,review_id,review_body,star_rating
0,R3B4Z83QSCNAU9,I couldn't have asked for a better purchasing ...,5
1,R28BUCHXGP7MAG,Thank god i did not pay $129.00 for this guita...,1
2,R221RYSJIR32JS,Its an okay clip. I have had better. It does n...,2
3,R3LUFVVZFGDXDW,Well I ordered these when I ordered my stand j...,2
4,R1VBOMACX4PUCO,This is something that is necessary if you are...,5
5,R18P91UP19K2TL,"This is a very poor quality guitar, and I am e...",1
6,RMJNX41B229DF,I'm very pleased with this recorder. One of th...,4
7,R24S6X2NTSB9S,"I bought these on the basis of the reviews, th...",1
8,ROP5SUKM14W4M,"I really, really enjoy these. The sound is sm...",5
9,R3N2B9BUZW8S29,An excellent product that performs as advertis...,5


### Preview Feature를 사용하기 위해  `AmazonAthenaPreviewFunctionality` Work Group에 추가합니다. 

In [35]:
import boto3
from botocore.exceptions import ClientError

client = boto3.client('athena')

try:
    response = client.create_work_group(Name='AmazonAthenaPreviewFunctionality') 
    print(response)
except ClientError as e:
    if e.response['Error']['Code'] == 'InvalidRequestException':
        print("Workgroup already exists.")
    else:
        print("Unexpected error: %s" % e)
    


Workgroup already exists.


# SQL Query 생성하기

`USING FUNCTION`절은 Athena 함수(preview) 또는 쿼리 내 후속 `SELECT` 문에서 참조할 수 있는 여러 함수에서 ML을 지정하여 사용합니다. 변수와 return 값에 대한 데이터 타입과 변수명, 함수명을 정의합니다.

In [36]:
statement = """
USING FUNCTION predict_star_rating(review_body VARCHAR) 
    RETURNS VARCHAR TYPE
    SAGEMAKER_INVOKE_ENDPOINT WITH (sagemaker_endpoint = '{}'
)
SELECT review_id, review_body, star_rating, predict_star_rating(REPLACE(review_body, ',', ' ')) AS predicted_star_rating 
    FROM {}.{} LIMIT 10
    """.format(autopilot_endpoint_name, database_name, table_name)

print(statement)


USING FUNCTION predict_star_rating(review_body VARCHAR) 
    RETURNS VARCHAR TYPE
    SAGEMAKER_INVOKE_ENDPOINT WITH (sagemaker_endpoint = 'automl-dm-ep-19-01-33-55'
)
SELECT review_id, review_body, star_rating, predict_star_rating(REPLACE(review_body, ',', ' ')) AS predicted_star_rating 
    FROM awsdb_1124.product_reviews LIMIT 10
    


In [37]:
# Execute statement using connection cursor
cursor = connect(region_name=region_name, 
                 s3_staging_dir=s3_staging_dir).cursor()
cursor.execute(statement, 
               work_group='AmazonAthenaPreviewFunctionality')

In [38]:
df = as_pandas(cursor)
df

,review_id,review_body,star_rating,predicted_star_rating
0,R3B4Z83QSCNAU9,I couldn't have asked for a better purchasing ...,5,5
1,R28BUCHXGP7MAG,Thank god i did not pay $129.00 for this guita...,1,1
2,R221RYSJIR32JS,Its an okay clip. I have had better. It does n...,2,3
3,R3LUFVVZFGDXDW,Well I ordered these when I ordered my stand j...,2,3
4,R1VBOMACX4PUCO,This is something that is necessary if you are...,5,3
5,R18P91UP19K2TL,"This is a very poor quality guitar, and I am e...",1,2
6,RMJNX41B229DF,I'm very pleased with this recorder. One of th...,4,3
7,R24S6X2NTSB9S,"I bought these on the basis of the reviews, th...",1,5
8,ROP5SUKM14W4M,"I really, really enjoy these. The sound is sm...",5,5
9,R3N2B9BUZW8S29,An excellent product that performs as advertis...,5,5


In [39]:
# sm.delete_endpoint(
#     EndpointName=autopilot_endpoint_name
# )

In [40]:
%store

Stored variables and their in-db values:
artifacts_dir                       -> 's3://sagemaker-us-east-1-322537213286/training-jo
comprehend_endpoint_arn             -> 'arn:aws:comprehend:us-east-1:322537213286:documen
data_bucket                         -> 'sagemaker-us-east-1-322537213286'
database_name                       -> 'awsdb_1124'
header_train_s3_uri                 -> 's3://sagemaker-us-east-1-322537213286/data/amazon
hyperparameters                     -> {'model_name': 'resnet50', 'height': 128, 'width':
job_bucket                          -> 'sagemaker-experiments-us-east-1-322537213286'
model_dir                           -> './model'
noheader_train_s3_uri               -> 's3://sagemaker-us-east-1-322537213286/data/amazon
output_dir                          -> './output'
region_name                         -> 'us-east-1'
role                                -> 'arn:aws:iam::322537213286:role/service-role/AIMLW
s3_destination_path_tsv             -> 's3://sagemaker-us